<a href="https://colab.research.google.com/github/newtonfalbo/Machine-Learning/blob/main/sistema_recomendacao_imagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import faiss
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Criando o modelo base para extrair características
def build_base_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu')
    ])
    return model

# Criando a Siamese Network
def build_siamese_network():
    base_model = build_base_model()

    input_a = tf.keras.Input(shape=(128, 128, 3))
    input_b = tf.keras.Input(shape=(128, 128, 3))

    encoded_a = base_model(input_a)
    encoded_b = base_model(input_b)

    # Distância entre as representações
    distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([encoded_a, encoded_b])
    output = layers.Dense(1, activation='sigmoid')(distance)

    model = Model(inputs=[input_a, input_b], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Criando e exibindo o modelo
siamese_model = build_siamese_network()
siamese_model.summary()

# Carregar imagens de um diretório
def load_images_from_folder(folder, target_size=(128, 128)):
    images = []
    image_names = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img) / 255.0  # Normaliza os pixels para [0,1]
        images.append(img_array)
        image_names.append(filename)
    return np.array(images), image_names

# Extração de embeddings
def extract_embeddings(model, images):
    base_model = Model(inputs=model.input[0], outputs=model.layers[-3].output)
    return base_model.predict(images)

# Criando banco de dados FAISS para busca de imagens similares
def build_faiss_index(embeddings):
    d = embeddings.shape[1]  # Dimensão do vetor de características
    index = faiss.IndexFlatL2(d)
    index.add(embeddings)
    return index

# Buscando imagens similares
def search_similar_images(index, query_embedding, k=5):
    distances, indices = index.search(query_embedding, k)
    return indices

# Exemplo de uso
image_folder = "caminho/para/as/imagens"
images, image_names = load_images_from_folder(image_folder)
embeddings = extract_embeddings(siamese_model, images)
faiss_index = build_faiss_index(embeddings)

# Testando com uma nova imagem
query_image = images[0].reshape(1, 128, 128, 3)
query_embedding = extract_embeddings(siamese_model, query_image)
similar_indices = search_similar_images(faiss_index, query_embedding)
print("Imagens mais similares:", [image_names[i] for i in similar_indices[0]])